In [4]:
target=['富蘭克林','安聯','聯博','摩根','元大','國泰']

import matplotlib.pyplot as plt
import pandas as pd
import requests
from lxml import etree 
from bs4 import BeautifulSoup
import time

# ----------------------------------------------------------------------------------------------
# 取得目標公司基本資訊
print('取得目標公司基本資訊...')


res=requests.get('https://announce.fundclear.com.tw/MOPSonshoreFundWeb/B01_02.jsp',
                headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"})

if str(res) == '<Response [200]>':
    print('get web data successfully')
else:
    print('Failed to get web data')

soup = BeautifulSoup(res.text, 'html.parser')


firm=[]
X1 = True
i = 0
while X1==True:
    for j in range(1,3):
        try:
            Firmlist=soup.select(f'.row{j}')[i]
            temp = {
                'ID': Firmlist.select('td')[0].get_text(),
                'Name': Firmlist.select('td')[1].get_text(),
            }
            firm.append(temp)
        except Exception as Err:
            X1=False
            print('firm data are all Done', Err)
    i+=1

# ----------------------------------------------------------------------------------------------
# 取得目標公司產品總覽    
print('取得目標公司產品總覽 ...')
time.sleep(3)

firmAhref=[]
for x in firm:
    ID=x['ID']
    for tar in target:
        if tar in x['Name']:
            print(f'https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent={ID}&type=2&XXX=null&agent={ID}&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=1')
            firmAhref.append({ 'Ahref': f'https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent={ID}&type=2&XXX=null&agent={ID}&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=',
                             'Agent':ID
                             })

#-----------------------------------------------------------------------------------------------------------------            
# 取得各產品資訊  
print('取得各產品資訊...')
time.sleep(3)



fundlist=[]
number=0

for tar_firm in firmAhref:
    X2=True
    page=1
    while X2==True:
        res=requests.get(list(tar_firm.values())[0]+f'{page}',headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"})
        soup = BeautifulSoup(res.text, 'html.parser')
        for i in range(10):
            for j in range(1,3):
                try:
                    a=soup.select(f'.row{j}')[i]
                    temp = {
                        'Index':number,
                        'Agent':list(tar_firm.values())[1],
                        'ID': a.select('td')[0].get_text(),
                        'Name': a.select('td')[1].get_text().split('\r')[0],
                        'Type':a.select('td')[2].get_text(),
                        'tendency':a.select('td')[3].get_text(),        
                        'Currency': a.select('td')[4].get_text(),
                        'Size(NTD)': a.select('td')[5].get_text(),
                        'Size(Orginal)': a.select('td')[6].get_text(),
                        'Proportion(%)': a.select('td')[7].get_text(),
                        'Net Value Date': a.select('td')[8].get_text(),
                        'Net Value': a.select('td')[9].get_text(),
                        'Amplitude(since 1/1)': a.select('td')[10].get_text(),
                        'Amplitude(near 3M)': a.select('td')[11].get_text(),
                        'Amplitude(near 6M)': a.select('td')[12].get_text(),
                        'Amplitude(near 1Y)': a.select('td')[13].get_text(),
                        'Amplitude(near 3Y)': a.select('td')[14].get_text(),
                        'Amplitude(near 5Y)': a.select('td')[15].get_text()   
                    }
                    fundlist.append(temp)
                    print(a.select('td')[0].get_text(),a.select('td')[1].get_text().split('\r')[0],' is done')
                    number+=1
                except Exception as Err:
                    print('Ended ', Err)
                    X2=False
        print(' page',page,'is done')
        page+=1



columns = [
    'Index',
    'Agent',
    'ID',
    'Name',
    'Type',
    'tendency',
    'Currency',
    'Size(NTD)',
    'Size(Orginal)',
    'Proportion(%)',
    'Net Value Date',
    'Net Value',
    'Amplitude(since 1/1)',
    'Amplitude(near 3M)',
    'Amplitude(near 6M)',
    'Amplitude(near 1Y)',
    'Amplitude(near 3Y)',
    'Amplitude(near 5Y)']

df_main = pd.DataFrame(fundlist,columns=columns)

print('loading detial data...')
time.sleep(3)


detial=[]
num2=1
for fundId in df_main['ID']:
    res=requests.get(f'https://announce.fundclear.com.tw/MOPSonshoreFundWeb/main1.jsp?fundId={fundId}',
                     headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"})
    soup=BeautifulSoup(res.text,'html.parser')
    temp2 = {
        'Manager': soup.select('.FieldContent')[19].get_text(),
        'Domestic Custodian Bank': soup.select('.FieldContent')[20].get_text(),
        'Foreign Custodian Bank': soup.select('.FieldContent')[21].get_text(),
        'Foreign Depository institution': soup.select('.FieldContent')[22].get_text(),
        'Risk Level': soup.select('.FieldContent')[24].get_text(),
        'Pattern': soup.select('.FieldContent')[25].get_text(),
        'Category': soup.select('.FieldContent')[26].get_text(),
        'Riegon': soup.select('.FieldContent')[27].get_text(),
        'Investment Targets': soup.select('.FieldContent')[28].get_text(),
        'Fund Renovation Regulations': soup.select('.FieldContent')[29].get_text(),
        'Fund Income Distribution Regulations': soup.select('.FieldContent')[30].get_text(),
        'Update Date': soup.select('.FieldContent')[31].get_text()
    }
    detial.append(temp2)
    num2+=1
    print(f'ID:{fundId} is done {num2}/{len(df_main)}' )


detial_columns = [
    'Manager',
    'Domestic Custodian Bank',
    'Foreign Custodian Bank',
    'Foreign Depository institution',
    'Risk Level',
    'Pattern',
    'Category',
    'Riegon',
    'Investment Targets',
    'Fund Renovation Regulations',
    'Fund Income Distribution Regulations',
    'Update Date']

# ------------------------------------------------------------------
# 取得費用率、報酬率
print('取得費用率、報酬率...')
time.sleep(3)

td=[]
columns_1=[]
expense=[]
returned=[]

for i in range(len(df_main)+1):
    try:
        a=df_main['Agent'][i]
        b=df_main['ID'][i]

        try:
            url=f'https://announce.fundclear.com.tw/MOPSonshoreFundWeb/INQ811.jsp?qSelect=S&checkSubmit=Y&agentId={a}&fundId={b}'
            headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}
            res = requests.get(url, headers = headers)
            soup=BeautifulSoup(res.text,'html.parser')
            rate_table=soup.select('.row1 td')

            c = {
                'ID':str(list(rate_table)[0]).split('>')[1].split('<')[0],
                'Name':str(list(rate_table)[1]).split('>')[1].split('\r')[0],
                '2017':str(list(rate_table)[2]).split('>')[1].split('<')[0],
                '2018':str(list(rate_table)[3]).split('>')[1].split('<')[0],
                '2019':str(list(rate_table)[4]).split('>')[1].split('<')[0],
                '2020':str(list(rate_table)[5]).split('>')[1].split('<')[0],
                '2021':str(list(rate_table)[6]).split('>')[1].split('<')[0],
                }    

            expense.append(c)
            print('ID',b ,' expense rate data is done')

        except:
            print(i,'expense')
    except:
        print('Expense Finish !')

for i in range(len(df_main)+1):
    try:
        a=df_main['Agent'][i]
        b=df_main['ID'][i]        
        try:
            url=f'https://announce.fundclear.com.tw/MOPSonshoreFundWeb/INQ811_1.jsp?qSelect=S&checkSubmit=Y&agentId={a}&fundId={b}'
            headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}
            res = requests.get(url, headers = headers)
            soup=BeautifulSoup(res.text,'html.parser')
            rate_table=soup.select('.row1 td')

            d = {
                'ID':str(list(rate_table)[0]).split('>')[1].split('<')[0],
                'Name':str(list(rate_table)[1]).split('>')[1].split('\r')[0],
                '2017':str(list(rate_table)[2]).split('>')[1].split('<')[0],
                '2018':str(list(rate_table)[3]).split('>')[1].split('<')[0],
                '2019':str(list(rate_table)[4]).split('>')[1].split('<')[0],
                '2020':str(list(rate_table)[5]).split('>')[1].split('<')[0],
                '2021':str(list(rate_table)[6]).split('>')[1].split('<')[0],
                }    

            returned.append(d)
            print('ID:',b ,' returned rate data is done')

        except:
            print(i,'returned')
    except:
        print('Returned Finish !')

uni_columns=['ID','Name','2017','2018','2019','2020','2021']

df_expense=pd.DataFrame(expense,columns=uni_columns)
df_returned=pd.DataFrame(returned,columns=uni_columns)


# --------------------------------------------------------------------
# 匯出檔案
# print('匯出檔案...')

# expense_file_name='費用率.csv'
# df_expense.to_csv(expense_file_name,encoding='utf-8')
# print(f'Export file name: {expense_file_name}')

# returned_file_name='報酬率.csv'
# df_returned.to_csv(returned_file_name,encoding='utf-8')
# print(f'Export file name: {returned_file_name}')

# df_detial = pd.DataFrame(detial,columns=detial_columns)

# df_main=df_main.join(df_detial, how='left')

# file_name='基金觀測站爬蟲.csv'
# df_main.to_csv(file_name,encoding='utf-8')
# print(f'Export file name: {file_name}')

# print('All done!!!')

取得目標公司基本資訊...
get web data successfully
firm data are all Done list index out of range
取得目標公司產品總覽 ...
https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent=A0011&type=2&XXX=null&agent=A0011&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=1
https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent=A0018&type=2&XXX=null&agent=A0018&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=1
https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent=A0037&type=2&XXX=null&agent=A0037&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=1
https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent=A0045&type=2&XXX=null&agent=A0045&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=1
https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent=A0005&type=2&XXX=null&agent=A0005&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=1
https://announce.fundclear.com.tw/MOPS

26317735C 國泰中國內需增長基金-美元I  is done
26317735D 國泰中國內需增長基金-人民幣  is done
26331835A 國泰新興高收益債券基金-新台幣A(不配息)  is done
26331835B 國泰新興高收益債券基金-新台幣B(配息)  is done
26331835C 國泰新興高收益債券基金-美元A(不配息)  is done
26331835D 國泰新興高收益債券基金-美元B(配息)  is done
26331835E 國泰新興高收益債券基金-人民幣A(不配息)  is done
26331835F 國泰新興高收益債券基金-人民幣B(配息)  is done
26331835G 國泰新興高收益債券基金-美元I(不配息)  is done
31821137A 國泰中國新興戰略基金-新台幣  is done
31821137B 國泰中國新興戰略基金-美元  is done
31821137C 國泰中國新興戰略基金-人民幣  is done
38482908A 國泰人民幣貨幣市場基金-人民幣  is done
38482908B 國泰人民幣貨幣市場基金-美元  is done
38482914A 國泰中國新興債券基金-新台幣  is done
38482914B 國泰中國新興債券基金-美元  is done
38482914C 國泰中國新興債券基金-人民幣  is done
38582842A 國泰全球多重收益平衡基金-新台幣A(不配息)  is done
38582842B 國泰全球多重收益平衡基金-新台幣B(配息)  is done
38582842C 國泰全球多重收益平衡基金-美元A(不配息)  is done
 page 2 is done
38582842D 國泰全球多重收益平衡基金-美元B(配息型)  is done
38582842E 國泰全球多重收益平衡基金-美元I(不配息)  is done
38582842F 國泰全球多重收益平衡基金-澳幣A(不配息)  is done
38582842G 國泰全球多重收益平衡基金-新台幣NB(配息型)  is done
38582842H 國泰全球多重收益平衡基金-美元NB(配息型)  is done
38582842I 國泰全球多重收益平衡基金-新台幣R(不配息)

42407614C 富蘭克林華美亞太平衡基金-美元(累積型)  is done
42407614D 富蘭克林華美亞太平衡基金-美元(分配型)  is done
42407614E 富蘭克林華美亞太平衡基金-人民幣(分配型)  is done
42483963A 富蘭克林華美策略高股息基金-新台幣累積型  is done
42483963B 富蘭克林華美策略高股息基金-新台幣分配型  is done
42483963C 富蘭克林華美策略高股息基金-美元累積型  is done
42483963D 富蘭克林華美策略高股息基金-美元分配型  is done
42539713A 富蘭克林華美退休傘型之目標2027組合基金-新台幣  is done
42539713B 富蘭克林華美退休傘型之目標2027組合基金-美元  is done
42539713C 富蘭克林華美退休傘型之目標2027組合基金-新台幣R類型  is done
42539729A 富蘭克林華美退休傘型之目標2037組合基金-新台幣  is done
42539729B 富蘭克林華美退休傘型之目標2037組合基金-美元  is done
42539729C 富蘭克林華美退休傘型之目標2037組合基金-新台幣R類型  is done
42539734A 富蘭克林華美退休傘型之目標2047組合基金-新台幣  is done
42539734B 富蘭克林華美退休傘型之目標2047組合基金-美元  is done
42539734C 富蘭克林華美退休傘型之目標2047組合基金-新台幣R類型  is done
42547333A 富蘭克林華美全球醫療保健基金-新台幣  is done
42547333B 富蘭克林華美全球醫療保健基金-美元  is done
42547333C 富蘭克林華美全球醫療保健基金-新台幣N類型  is done
42547333D 富蘭克林華美全球醫療保健基金-美元N類型  is done
 page 4 is done
48858299A 富蘭克林華美坦伯頓全球股票組合基金-A類型  is done
48858299B 富蘭克林華美坦伯頓全球股票組合基金-R類型  is done
48997270A 富蘭克林華美新世界股票基金-新臺幣  is done
48997270B 富蘭克林華美新世界

42319470B 安聯中華新思路基金-人民幣  is done
42319470C 安聯中華新思路基金-美元  is done
48806595 安聯全球綠能趨勢基金  is done
48937267A 安聯全球人口趨勢基金-A類型-新臺幣  is done
48937267B 安聯全球人口趨勢基金-R類型-新臺幣  is done
48997911 安聯台灣智慧基金  is done
48981946 安聯全球農金趨勢基金  is done
72954167A 安聯收益成長多重資產基金-A類型(累積)-新臺幣  is done
72954167B 安聯收益成長多重資產基金-B類型(月配息)-新臺幣  is done
72954167C 安聯收益成長多重資產基金-A類型(累積)-人民幣  is done
72954167D 安聯收益成長多重資產基金-B類型(月配息)-人民幣  is done
72954167E 安聯收益成長多重資產基金- N類型(月配息)-新臺幣  is done
72954167F 安聯收益成長多重資產基金- N類型(月配息)-人民幣  is done
72954167G 安聯收益成長多重資產基金-N類型(月配息)-美元  is done
72954167H 安聯收益成長多重資產基金-A類型(累積)-美元  is done
72954167I 安聯收益成長多重資產基金-B類型(月配息)-美元  is done
72954167J 安聯收益成長多重資產基金- C類型(累積)-新臺幣  is done
72954167K 安聯收益成長多重資產基金- C類型(累積)-美元  is done
74711899I 安聯美國短年期高收益債券基金- A類型(累積)-人民幣  is done
74711899A 安聯美國短年期高收益債券基金-A類型(累積)-新臺幣  is done
 page 3 is done
74711899B 安聯美國短年期高收益債券基金-B類型(月配息)-新臺幣  is done
74711899C 安聯美國短年期高收益債券基金- N類型(月配息)-新臺幣  is done
74711899D 安聯美國短年期高收益債券基金- B類型(月配息)-人民幣  is done
74711899E 安聯美國短年期高收益債券基金- N類型(月配

ID:42331684C is done 196/626
ID:42331684D is done 197/626
ID:42331684E is done 198/626
ID:42331684F is done 199/626
ID:42331684A is done 200/626
ID:42385127 is done 201/626
ID:42385148 is done 202/626
ID:42389475 is done 203/626
ID:42401610A is done 204/626
ID:42401610B is done 205/626
ID:42401610C is done 206/626
ID:42401610D is done 207/626
ID:42404258 is done 208/626
ID:42404264 is done 209/626
ID:42443203 is done 210/626
ID:42443225 is done 211/626
ID:42470989C is done 212/626
ID:42470989D is done 213/626
ID:42470989A is done 214/626
ID:42470989B is done 215/626
ID:42482726 is done 216/626
ID:42482731 is done 217/626
ID:42482747 is done 218/626
ID:42530608 is done 219/626
ID:42530591 is done 220/626
ID:48681504C is done 221/626
ID:48681504D is done 222/626
ID:48681504E is done 223/626
ID:48681504A is done 224/626
ID:48681504B is done 225/626
ID:48832881A is done 226/626
ID:48832881B is done 227/626
ID:72939309 is done 228/626
ID:72939315 is done 229/626
ID:72939321 is done 230/626


...

ID 25687674E  expense rate data is done
ID 25687674F  expense rate data is done
ID 26287795A  expense rate data is done
ID 26287795B  expense rate data is done
ID 26287795C  expense rate data is done
ID 26310297  expense rate data is done
ID 31836244A  expense rate data is done
ID 31836244B  expense rate data is done
ID 31836244C  expense rate data is done
ID 31836244D  expense rate data is done
ID 31836244E  expense rate data is done
ID 31836244F  expense rate data is done
ID 31836244G  expense rate data is done
ID 31836244H  expense rate data is done
ID 31836244I  expense rate data is done
ID 31836244J  expense rate data is done
ID 31836244K  expense rate data is done
ID 31836244L  expense rate data is done
ID 31836244M  expense rate data is done
ID 31836244N  expense rate data is done
ID 31836244O  expense rate data is done
ID 31836244P  expense rate data is done
ID 31836244Q  expense rate data is done
ID 31897044  expense rate data is done
ID 31920832A  expense rate data is done
ID

ID 77549343  expense rate data is done
ID 81584335  expense rate data is done
ID 83863951  expense rate data is done
ID 92017204  expense rate data is done
ID 92026219  expense rate data is done
ID 92001765  expense rate data is done
ID 98764389  expense rate data is done
ID 99346928  expense rate data is done
ID 99967974A  expense rate data is done
ID 99967974B  expense rate data is done
ID 00974066  expense rate data is done
ID 01032058  expense rate data is done
ID 10348058  expense rate data is done
ID 13530495  expense rate data is done
ID 14693593A  expense rate data is done
ID 14693593B  expense rate data is done
ID 17604124A  expense rate data is done
ID 17604124B  expense rate data is done
ID 18033003  expense rate data is done
ID 18480065  expense rate data is done
ID 25589337A  expense rate data is done
ID 25589337B  expense rate data is done
ID 25589337C  expense rate data is done
ID 25627819  expense rate data is done
ID 25680150  expense rate data is done
ID 26398976A  ex

ID: 38522113R  returned rate data is done
ID: 38633201A  returned rate data is done
ID: 38633201B  returned rate data is done
ID: 38633201C  returned rate data is done
ID: 38633201D  returned rate data is done
ID: 38633201E  returned rate data is done
ID: 38633201F  returned rate data is done
ID: 38633201G  returned rate data is done
ID: 38633217A  returned rate data is done
ID: 38633217B  returned rate data is done
ID: 38633217C  returned rate data is done
ID: 38654486A  returned rate data is done
ID: 38654486B  returned rate data is done
ID: 38654486C  returned rate data is done
ID: 38654486D  returned rate data is done
ID: 38654486E  returned rate data is done
ID: 38654486F  returned rate data is done
ID: 38654486G  returned rate data is done
ID: 38654486H  returned rate data is done
ID: 38654486I  returned rate data is done
ID: 38654486J  returned rate data is done
ID: 38654486K  returned rate data is done
ID: 38654486L  returned rate data is done
ID: 38654486M  returned rate data 

ID: 88090913  returned rate data is done
ID: 88090934A  returned rate data is done
ID: 88090934B  returned rate data is done
ID: 88114133  returned rate data is done
ID: 88185256  returned rate data is done
ID: 88237803  returned rate data is done
ID: 88259257A  returned rate data is done
ID: 88259257B  returned rate data is done
ID: 01031874B  returned rate data is done
ID: 01031874C  returned rate data is done
ID: 15733973  returned rate data is done
ID: 25515602  returned rate data is done
ID: 21718768A  returned rate data is done
ID: 21718768B  returned rate data is done
ID: 21718768C  returned rate data is done
ID: 21718768D  returned rate data is done
ID: 25687668  returned rate data is done
ID: 25687674A  returned rate data is done
ID: 25687674B  returned rate data is done
ID: 25687674C  returned rate data is done
ID: 25687674D  returned rate data is done
ID: 25687674E  returned rate data is done
ID: 25687674F  returned rate data is done
ID: 26287795A  returned rate data is done

ID: 48903175  returned rate data is done
ID: 48903180  returned rate data is done
ID: 48903196  returned rate data is done
ID: 48885346A  returned rate data is done
ID: 48885346B  returned rate data is done
ID: 48952816A  returned rate data is done
ID: 48952816B  returned rate data is done
ID: 48966213  returned rate data is done
ID: 72935887A  returned rate data is done
ID: 72935887B  returned rate data is done
ID: 72935887C  returned rate data is done
ID: 72963026  returned rate data is done
ID: 73656187  returned rate data is done
ID: 73694555  returned rate data is done
ID: 74705236  returned rate data is done
ID: 74722571  returned rate data is done
ID: 74722592  returned rate data is done
ID: 74722657  returned rate data is done
ID: 75970921A  returned rate data is done
ID: 75970921B  returned rate data is done
ID: 75970963A  returned rate data is done
ID: 75970963B  returned rate data is done
ID: 75971153A  returned rate data is done
ID: 75971153B  returned rate data is done
ID:

In [9]:
df_main.columns

Index(['Index', 'Agent', 'ID', 'Name', 'Type', 'tendency', 'Currency',
       'Size(NTD)', 'Size(Orginal)', 'Proportion(%)', 'Net Value Date',
       'Net Value', 'Amplitude(since 1/1)', 'Amplitude(near 3M)',
       'Amplitude(near 6M)', 'Amplitude(near 1Y)', 'Amplitude(near 3Y)',
       'Amplitude(near 5Y)'],
      dtype='object')

# function

In [23]:
# target=['富蘭克林','安聯','聯博','摩根','元大','國泰']

def fundcollect(*args):
    target=[]
    for num in args:
        target.append(num)

    import matplotlib.pyplot as plt
    import pandas as pd
    import requests
    from lxml import etree 
    from bs4 import BeautifulSoup
    import time

    # ----------------------------------------------------------------------------------------------
    # 取得目標公司基本資訊
    print('取得目標公司基本資訊...')
    time.sleep(3)



    res=requests.get('https://announce.fundclear.com.tw/MOPSonshoreFundWeb/B01_02.jsp',
                    headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"})

    if str(res) == '<Response [200]>':
        print('get web data successfully')
    else:
        print('Failed to get web data')

    soup = BeautifulSoup(res.text, 'html.parser')


    firm=[]
    X1 = True
    i = 0
    while X1==True:
        for j in range(1,3):
            try:
                Firmlist=soup.select(f'.row{j}')[i]
                temp = {
                    'ID': Firmlist.select('td')[0].get_text(),
                    'Name': Firmlist.select('td')[1].get_text(),
                }
                firm.append(temp)
            except Exception as Err:
                X1=False
                print('firm data are all Done', Err)
        i+=1

    # ----------------------------------------------------------------------------------------------
    # 取得目標公司產品總覽    
    print('取得目標公司產品總覽 ...')
    time.sleep(3)


    firmAhref=[]
    for x in firm:
        ID=x['ID']
        for tar in target:
            if tar in x['Name']:
                print(f'https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent={ID}&type=2&XXX=null&agent={ID}&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=1')
                firmAhref.append({ 'Ahref': f'https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent={ID}&type=2&XXX=null&agent={ID}&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=',
                                 'Agent':ID
                                 })

    #-----------------------------------------------------------------------------------------------------------------            
    # 取得各產品資訊  
    print('取得各產品資訊...')



    fundlist=[]
    number=0

    for tar_firm in firmAhref:
        X2=True
        page=1
        while X2==True:
            res=requests.get(list(tar_firm.values())[0]+f'{page}',headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"})
            soup = BeautifulSoup(res.text, 'html.parser')
            for i in range(10):
                for j in range(1,3):
                    try:
                        a=soup.select(f'.row{j}')[i]
                        temp = {
                            'Index':number,
                            'Agent':list(tar_firm.values())[1],
                            'ID': a.select('td')[0].get_text(),
                            'Name': a.select('td')[1].get_text().split('\r')[0],
                            'Type':a.select('td')[2].get_text(),
                            'tendency':a.select('td')[3].get_text(),        
                            'Currency': a.select('td')[4].get_text(),
                            'Size(NTD)': a.select('td')[5].get_text(),
                            'Size(Orginal)': a.select('td')[6].get_text(),
                            'Proportion(%)': a.select('td')[7].get_text(),
                            'Net Value Date': a.select('td')[8].get_text(),
                            'Net Value': a.select('td')[9].get_text(),
                            'Amplitude(since 1/1)': a.select('td')[10].get_text(),
                            'Amplitude(near 3M)': a.select('td')[11].get_text(),
                            'Amplitude(near 6M)': a.select('td')[12].get_text(),
                            'Amplitude(near 1Y)': a.select('td')[13].get_text(),
                            'Amplitude(near 3Y)': a.select('td')[14].get_text(),
                            'Amplitude(near 5Y)': a.select('td')[15].get_text()   
                        }
                        fundlist.append(temp)
                        print(a.select('td')[0].get_text(),a.select('td')[1].get_text().split('\r')[0],' is done')
                        number+=1
                    except Exception as Err:
                        print('Ended ', Err)
                        X2=False
            print(' page',page,'is done')
            page+=1



    columns = [
        'Index',
        'Agent',
        'ID',
        'Name',
        'Type',
        'tendency',
        'Currency',
        'Size(NTD)',
        'Size(Orginal)',
        'Proportion(%)',
        'Net Value Date',
        'Net Value',
        'Amplitude(since 1/1)',
        'Amplitude(near 3M)',
        'Amplitude(near 6M)',
        'Amplitude(near 1Y)',
        'Amplitude(near 3Y)',
        'Amplitude(near 5Y)']

    df_main = pd.DataFrame(fundlist,columns=columns)

    print('loading detial data...')
    time.sleep(3)


    detial=[]
    num2=1
    for fundId in df_main['ID']:
        res=requests.get(f'https://announce.fundclear.com.tw/MOPSonshoreFundWeb/main1.jsp?fundId={fundId}',
                         headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"})
        soup=BeautifulSoup(res.text,'html.parser')
        temp2 = {
            'Manager': soup.select('.FieldContent')[19].get_text(),
            'Domestic Custodian Bank': soup.select('.FieldContent')[20].get_text(),
            'Foreign Custodian Bank': soup.select('.FieldContent')[21].get_text(),
            'Foreign Depository institution': soup.select('.FieldContent')[22].get_text(),
            'Risk Level': soup.select('.FieldContent')[24].get_text(),
            'Pattern': soup.select('.FieldContent')[25].get_text(),
            'Category': soup.select('.FieldContent')[26].get_text(),
            'Riegon': soup.select('.FieldContent')[27].get_text(),
            'Investment Targets': soup.select('.FieldContent')[28].get_text(),
            'Fund Renovation Regulations': soup.select('.FieldContent')[29].get_text(),
            'Fund Income Distribution Regulations': soup.select('.FieldContent')[30].get_text(),
            'Update Date': soup.select('.FieldContent')[31].get_text()
        }
        detial.append(temp2)
        num2+=1
        print(f'ID:{fundId} is done {num2}/{len(df_main)}' )


    detial_columns = [
        'Manager',
        'Domestic Custodian Bank',
        'Foreign Custodian Bank',
        'Foreign Depository institution',
        'Risk Level',
        'Pattern',
        'Category',
        'Riegon',
        'Investment Targets',
        'Fund Renovation Regulations',
        'Fund Income Distribution Regulations',
        'Update Date']

    # ------------------------------------------------------------------
    # 取得費用率、報酬率
    print('取得費用率、報酬率...')
    time.sleep(3)

    td=[]
    columns_1=[]
    expense=[]
    returned=[]

    for i in range(len(df_main)+1):
        try:
            a=df_main['Agent'][i]
            b=df_main['ID'][i]

            try:
                url=f'https://announce.fundclear.com.tw/MOPSonshoreFundWeb/INQ811.jsp?qSelect=S&checkSubmit=Y&agentId={a}&fundId={b}'
                headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}
                res = requests.get(url, headers = headers)
                soup=BeautifulSoup(res.text,'html.parser')
                rate_table=soup.select('.row1 td')

                c = {
                    'ID':str(list(rate_table)[0]).split('>')[1].split('<')[0],
                    'Name':str(list(rate_table)[1]).split('>')[1].split('\r')[0],
                    '2017':str(list(rate_table)[2]).split('>')[1].split('<')[0],
                    '2018':str(list(rate_table)[3]).split('>')[1].split('<')[0],
                    '2019':str(list(rate_table)[4]).split('>')[1].split('<')[0],
                    '2020':str(list(rate_table)[5]).split('>')[1].split('<')[0],
                    '2021':str(list(rate_table)[6]).split('>')[1].split('<')[0],
                    }    

                expense.append(c)
                print('ID',b ,' expense rate data is done')

            except:
                print(i,'expense')
        except:
            print('Expense Finish !')

    for i in range(len(df_main)+1):
        try:
            a=df_main['Agent'][i]
            b=df_main['ID'][i]        
            try:
                url=f'https://announce.fundclear.com.tw/MOPSonshoreFundWeb/INQ811_1.jsp?qSelect=S&checkSubmit=Y&agentId={a}&fundId={b}'
                headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}
                res = requests.get(url, headers = headers)
                soup=BeautifulSoup(res.text,'html.parser')
                rate_table=soup.select('.row1 td')

                d = {
                    'ID':str(list(rate_table)[0]).split('>')[1].split('<')[0],
                    'Name':str(list(rate_table)[1]).split('>')[1].split('\r')[0],
                    '2017':str(list(rate_table)[2]).split('>')[1].split('<')[0],
                    '2018':str(list(rate_table)[3]).split('>')[1].split('<')[0],
                    '2019':str(list(rate_table)[4]).split('>')[1].split('<')[0],
                    '2020':str(list(rate_table)[5]).split('>')[1].split('<')[0],
                    '2021':str(list(rate_table)[6]).split('>')[1].split('<')[0],
                    }    

                returned.append(d)
                print('ID:',b ,' returned rate data is done')

            except:
                print(i,'returned')
        except:
            print('Returned Finish !')

    uni_columns=['ID','Name','2017','2018','2019','2020','2021']

    df_expense=pd.DataFrame(expense,columns=uni_columns)
    df_returned=pd.DataFrame(returned,columns=uni_columns)


    # --------------------------------------------------------------------
    # 匯出檔案
    print('匯出檔案...')
    time.sleep(3)


    expense_file_name='費用率.csv'
    df_expense.to_csv(expense_file_name,encoding='utf-8')
    print(f'Export file name: {expense_file_name}')

    returned_file_name='報酬率.csv'
    df_returned.to_csv(returned_file_name,encoding='utf-8')
    print(f'Export file name: {returned_file_name}')

    df_detial = pd.DataFrame(detial,columns=detial_columns)

    df_main=df_main.join(df_detial, how='left')

    file_name='基金觀測站爬蟲.csv'
    df_main.to_csv(file_name,encoding='utf-8')
    print(f'Export file name: {file_name}')

    print('All done!!!')

In [24]:
fundcollect('富蘭克林','安聯','聯博','摩根','元大','國泰')

取得目標公司基本資訊...
get web data successfully
firm data are all Done list index out of range
取得目標公司產品總覽 ...
https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent=A0011&type=2&XXX=null&agent=A0011&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=1
https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent=A0018&type=2&XXX=null&agent=A0018&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=1
https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent=A0037&type=2&XXX=null&agent=A0037&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=1
https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent=A0045&type=2&XXX=null&agent=A0045&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=1
https://announce.fundclear.com.tw/MOPSonshoreFundWeb/I01.jsp?agent=A0005&type=2&XXX=null&agent=A0005&fundtype=ALL&fundtype1=ALL&curr=ALL&sort=FND_ORGID&direction=ASC&pid=1
https://announce.fundclear.com.tw/MOPS

26317735C 國泰中國內需增長基金-美元I  is done
26317735D 國泰中國內需增長基金-人民幣  is done
26331835A 國泰新興高收益債券基金-新台幣A(不配息)  is done
26331835B 國泰新興高收益債券基金-新台幣B(配息)  is done
26331835C 國泰新興高收益債券基金-美元A(不配息)  is done
26331835D 國泰新興高收益債券基金-美元B(配息)  is done
26331835E 國泰新興高收益債券基金-人民幣A(不配息)  is done
26331835F 國泰新興高收益債券基金-人民幣B(配息)  is done
26331835G 國泰新興高收益債券基金-美元I(不配息)  is done
31821137A 國泰中國新興戰略基金-新台幣  is done
31821137B 國泰中國新興戰略基金-美元  is done
31821137C 國泰中國新興戰略基金-人民幣  is done
38482908A 國泰人民幣貨幣市場基金-人民幣  is done
38482908B 國泰人民幣貨幣市場基金-美元  is done
38482914A 國泰中國新興債券基金-新台幣  is done
38482914B 國泰中國新興債券基金-美元  is done
38482914C 國泰中國新興債券基金-人民幣  is done
38582842A 國泰全球多重收益平衡基金-新台幣A(不配息)  is done
38582842B 國泰全球多重收益平衡基金-新台幣B(配息)  is done
38582842C 國泰全球多重收益平衡基金-美元A(不配息)  is done
 page 2 is done
38582842D 國泰全球多重收益平衡基金-美元B(配息型)  is done
38582842E 國泰全球多重收益平衡基金-美元I(不配息)  is done
38582842F 國泰全球多重收益平衡基金-澳幣A(不配息)  is done
38582842G 國泰全球多重收益平衡基金-新台幣NB(配息型)  is done
38582842H 國泰全球多重收益平衡基金-美元NB(配息型)  is done
38582842I 國泰全球多重收益平衡基金-新台幣R(不配息)

42407614C 富蘭克林華美亞太平衡基金-美元(累積型)  is done
42407614D 富蘭克林華美亞太平衡基金-美元(分配型)  is done
42407614E 富蘭克林華美亞太平衡基金-人民幣(分配型)  is done
42483963A 富蘭克林華美策略高股息基金-新台幣累積型  is done
42483963B 富蘭克林華美策略高股息基金-新台幣分配型  is done
42483963C 富蘭克林華美策略高股息基金-美元累積型  is done
42483963D 富蘭克林華美策略高股息基金-美元分配型  is done
42539713A 富蘭克林華美退休傘型之目標2027組合基金-新台幣  is done
42539713B 富蘭克林華美退休傘型之目標2027組合基金-美元  is done
42539713C 富蘭克林華美退休傘型之目標2027組合基金-新台幣R類型  is done
42539729A 富蘭克林華美退休傘型之目標2037組合基金-新台幣  is done
42539729B 富蘭克林華美退休傘型之目標2037組合基金-美元  is done
42539729C 富蘭克林華美退休傘型之目標2037組合基金-新台幣R類型  is done
42539734A 富蘭克林華美退休傘型之目標2047組合基金-新台幣  is done
42539734B 富蘭克林華美退休傘型之目標2047組合基金-美元  is done
42539734C 富蘭克林華美退休傘型之目標2047組合基金-新台幣R類型  is done
42547333A 富蘭克林華美全球醫療保健基金-新台幣  is done
42547333B 富蘭克林華美全球醫療保健基金-美元  is done
42547333C 富蘭克林華美全球醫療保健基金-新台幣N類型  is done
42547333D 富蘭克林華美全球醫療保健基金-美元N類型  is done
 page 4 is done
48858299A 富蘭克林華美坦伯頓全球股票組合基金-A類型  is done
48858299B 富蘭克林華美坦伯頓全球股票組合基金-R類型  is done
48997270A 富蘭克林華美新世界股票基金-新臺幣  is done
48997270B 富蘭克林華美新世界

42319470B 安聯中華新思路基金-人民幣  is done
42319470C 安聯中華新思路基金-美元  is done
48806595 安聯全球綠能趨勢基金  is done
48937267A 安聯全球人口趨勢基金-A類型-新臺幣  is done
48937267B 安聯全球人口趨勢基金-R類型-新臺幣  is done
48997911 安聯台灣智慧基金  is done
48981946 安聯全球農金趨勢基金  is done
72954167A 安聯收益成長多重資產基金-A類型(累積)-新臺幣  is done
72954167B 安聯收益成長多重資產基金-B類型(月配息)-新臺幣  is done
72954167C 安聯收益成長多重資產基金-A類型(累積)-人民幣  is done
72954167D 安聯收益成長多重資產基金-B類型(月配息)-人民幣  is done
72954167E 安聯收益成長多重資產基金- N類型(月配息)-新臺幣  is done
72954167F 安聯收益成長多重資產基金- N類型(月配息)-人民幣  is done
72954167G 安聯收益成長多重資產基金-N類型(月配息)-美元  is done
72954167H 安聯收益成長多重資產基金-A類型(累積)-美元  is done
72954167I 安聯收益成長多重資產基金-B類型(月配息)-美元  is done
72954167J 安聯收益成長多重資產基金- C類型(累積)-新臺幣  is done
72954167K 安聯收益成長多重資產基金- C類型(累積)-美元  is done
74711899I 安聯美國短年期高收益債券基金- A類型(累積)-人民幣  is done
74711899A 安聯美國短年期高收益債券基金-A類型(累積)-新臺幣  is done
 page 3 is done
74711899B 安聯美國短年期高收益債券基金-B類型(月配息)-新臺幣  is done
74711899C 安聯美國短年期高收益債券基金- N類型(月配息)-新臺幣  is done
74711899D 安聯美國短年期高收益債券基金- B類型(月配息)-人民幣  is done
74711899E 安聯美國短年期高收益債券基金- N類型(月配

ID:42331684C is done 196/626
ID:42331684D is done 197/626
ID:42331684E is done 198/626
ID:42331684F is done 199/626
ID:42331684A is done 200/626
ID:42385127 is done 201/626
ID:42385148 is done 202/626
ID:42389475 is done 203/626
ID:42401610A is done 204/626
ID:42401610B is done 205/626
ID:42401610C is done 206/626
ID:42401610D is done 207/626
ID:42404258 is done 208/626
ID:42404264 is done 209/626
ID:42443203 is done 210/626
ID:42443225 is done 211/626
ID:42470989C is done 212/626
ID:42470989D is done 213/626
ID:42470989A is done 214/626
ID:42470989B is done 215/626
ID:42482726 is done 216/626
ID:42482731 is done 217/626
ID:42482747 is done 218/626
ID:42530608 is done 219/626
ID:42530591 is done 220/626
ID:48681504C is done 221/626
ID:48681504D is done 222/626
ID:48681504E is done 223/626
ID:48681504A is done 224/626
ID:48681504B is done 225/626
ID:48832881A is done 226/626
ID:48832881B is done 227/626
ID:72939309 is done 228/626
ID:72939315 is done 229/626
ID:72939321 is done 230/626


...

ID 25687674E  expense rate data is done
ID 25687674F  expense rate data is done
ID 26287795A  expense rate data is done
ID 26287795B  expense rate data is done
ID 26287795C  expense rate data is done
ID 26310297  expense rate data is done
ID 31836244A  expense rate data is done
ID 31836244B  expense rate data is done
ID 31836244C  expense rate data is done
ID 31836244D  expense rate data is done
ID 31836244E  expense rate data is done
ID 31836244F  expense rate data is done
ID 31836244G  expense rate data is done
ID 31836244H  expense rate data is done
ID 31836244I  expense rate data is done
ID 31836244J  expense rate data is done
ID 31836244K  expense rate data is done
ID 31836244L  expense rate data is done
ID 31836244M  expense rate data is done
ID 31836244N  expense rate data is done
ID 31836244O  expense rate data is done
ID 31836244P  expense rate data is done
ID 31836244Q  expense rate data is done
ID 31897044  expense rate data is done
ID 31920832A  expense rate data is done
ID

ID 77549343  expense rate data is done
ID 81584335  expense rate data is done
ID 83863951  expense rate data is done
ID 92017204  expense rate data is done
ID 92026219  expense rate data is done
ID 92001765  expense rate data is done
ID 98764389  expense rate data is done
ID 99346928  expense rate data is done
ID 99967974A  expense rate data is done
ID 99967974B  expense rate data is done
ID 00974066  expense rate data is done
ID 01032058  expense rate data is done
ID 10348058  expense rate data is done
ID 14693593A  expense rate data is done
ID 14693593B  expense rate data is done
ID 17604124A  expense rate data is done
ID 17604124B  expense rate data is done
ID 18033003  expense rate data is done
ID 18480065  expense rate data is done
ID 25589337A  expense rate data is done
ID 25589337B  expense rate data is done
ID 25589337C  expense rate data is done
ID 25627819  expense rate data is done
ID 25680150  expense rate data is done
ID 26398976A  expense rate data is done
ID 26398976B  e

ID: 38633201B  returned rate data is done
ID: 38633201C  returned rate data is done
ID: 38633201D  returned rate data is done
ID: 38633201E  returned rate data is done
ID: 38633201F  returned rate data is done
ID: 38633201G  returned rate data is done
ID: 38633217A  returned rate data is done
ID: 38633217B  returned rate data is done
ID: 38633217C  returned rate data is done
ID: 38654486A  returned rate data is done
ID: 38654486B  returned rate data is done
ID: 38654486C  returned rate data is done
ID: 38654486D  returned rate data is done
ID: 38654486E  returned rate data is done
ID: 38654486F  returned rate data is done
ID: 38654486G  returned rate data is done
ID: 38654486H  returned rate data is done
ID: 38654486I  returned rate data is done
ID: 38654486J  returned rate data is done
ID: 38654486K  returned rate data is done
ID: 38654486L  returned rate data is done
ID: 38654486M  returned rate data is done
ID: 38654486N  returned rate data is done
ID: 38654486O  returned rate data 

ID: 88090934A  returned rate data is done
ID: 88090934B  returned rate data is done
ID: 88114133  returned rate data is done
ID: 88185256  returned rate data is done
ID: 88237803  returned rate data is done
ID: 88259257A  returned rate data is done
ID: 88259257B  returned rate data is done
ID: 01031874B  returned rate data is done
ID: 01031874C  returned rate data is done
ID: 15733973  returned rate data is done
ID: 25515602  returned rate data is done
ID: 21718768A  returned rate data is done
ID: 21718768B  returned rate data is done
ID: 21718768C  returned rate data is done
ID: 21718768D  returned rate data is done
ID: 25687668  returned rate data is done
ID: 25687674A  returned rate data is done
ID: 25687674B  returned rate data is done
ID: 25687674C  returned rate data is done
ID: 25687674D  returned rate data is done
ID: 25687674E  returned rate data is done
ID: 25687674F  returned rate data is done
ID: 26287795A  returned rate data is done
ID: 26287795B  returned rate data is don

ID: 48903175  returned rate data is done
ID: 48903180  returned rate data is done
ID: 48903196  returned rate data is done
ID: 48885346A  returned rate data is done
ID: 48885346B  returned rate data is done
ID: 48952816A  returned rate data is done
ID: 48952816B  returned rate data is done
ID: 48966213  returned rate data is done
ID: 72935887A  returned rate data is done
ID: 72935887B  returned rate data is done
ID: 72935887C  returned rate data is done
ID: 72963026  returned rate data is done
ID: 73656187  returned rate data is done
ID: 73694555  returned rate data is done
ID: 74705236  returned rate data is done
ID: 74722571  returned rate data is done
ID: 74722592  returned rate data is done
ID: 74722657  returned rate data is done
ID: 75970921A  returned rate data is done
ID: 75970921B  returned rate data is done
ID: 75970963A  returned rate data is done
ID: 75970963B  returned rate data is done
ID: 75971153A  returned rate data is done
ID: 75971153B  returned rate data is done
ID: